In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

sys.path.append('..')
sys.path.append('../src')

import pandas as pd
import numpy as np

from functools import partial
import constants as cst
from loading import Loader
from preprocessing import Preprocessor
import seaborn as sns
from datetime import timedelta

In [ ]:
data_loader = Loader()
data = data_loader.load_data()

In [ ]:
preprocessor = Preprocessor()
preprocessor.preprocess(data)

# Loading data

In [ ]:
data = pd.read_csv(os.path.join('..', cst.TRAIN_TRANSACTION_PATH), index_col=0)

# Extract labels

In [ ]:
def purchase_frequency(dates):
    return pd.Timedelta(np.diff(dates.unique()).mean()).total_seconds()/(60*60*24)

In [ ]:
freqs = data.groupby(by="client_id", as_index=False).agg(
    last_purchase=('date_order', 'max'), 
    frequency=('date_order', purchase_frequency), 
)

In [ ]:
# define key dates
split_day = cst.MAX_DATE - timedelta(90)
freq_churn = split_day - timedelta(30)
medium_churn = split_day - timedelta(90)

In [ ]:
freqs.loc[(freqs['frequency'] > 0) & (freqs['frequency'] < 10), 'client_category'] = 'freq_buyer'
freqs.loc[(freqs['frequency'] >= 10) & (freqs['frequency'] < 32), 'client_category'] = 'med_buyer'
freqs.loc[(freqs['frequency'] >=32) | (freqs['frequency'] == 0), 'client_category'] = 'infreq_buyer'

In [ ]:
freqs.loc[(freqs['client_category']=='freq_buyer') & (freqs['last_purchase'] <= freq_churn), 'is_churn'] = 1
freqs.loc[(freqs['client_category']=='freq_buyer') & (freqs['last_purchase'] > freq_churn), 'is_churn'] = 0
freqs.loc[(freqs['client_category']=='med_buyer') & (freqs['last_purchase'] <= medium_churn), 'is_churn'] = 1
freqs.loc[(freqs['client_category']=='med_buyer') & (freqs['last_purchase'] > medium_churn), 'is_churn'] = 0

In [ ]:
freqs.to_csv(os.path.join('..', cst.TRAIN_TARGET_PATH))